In [1]:
pwd

'/Users/mdwivedi/Downloads/core-tech'

In [94]:
from datasets import Dataset, DatasetDict
import pandas as pd

In [2]:
# import torch
# from transformers import BertTokenizer, BertForPreTraining

In [3]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# model = BertForPreTraining.from_pretrained('bert-base-cased')

In [4]:
import pandas as pd
## Reading the dataframe 

data = pd.read_csv('due_dilligence_data.csv')
data.head()

,sentence,label,index,sent_id,topic_id
0,EXECUTION VERSION,B,0,150705_0,1252
1,THE BANK OF NOVA SCOTIA,B,1,150705_1,1252
2,"as Sole Bookrunner , Lead Arranger and Adminis...",B,2,150705_2,1252
3,- and-,B,3,150705_3,1252
4,BMO CAPITAL MARKETS,B,4,150705_4,1252


In [5]:
df = data[data['topic_id'] == 1469]
df.head()

,sentence,label,index,sent_id,topic_id
7909065,Exhibit 4.34,B,7909065,217889_0,1469
7909066,"€250,000,000 TERM FACILITY AGREEMENT",B,7909066,217889_1,1469
7909067,"€250,000,000",B,7909067,217889_2,1469
7909068,TERM FACILITY AGREEMENT,B,7909068,217889_3,1469
7909069,Dated 8 March 2017 for,B,7909069,217889_4,1469


In [6]:
df['label'] = df['label'].replace({'B':0, '1':1})
df.head()

/var/folders/ll/hqtjh2m5481_qkt_4p9g5v940000gn/T/ipykernel_46640/1221279264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].replace({'B':0, '1':1})


,sentence,label,index,sent_id,topic_id
7909065,Exhibit 4.34,0,7909065,217889_0,1469
7909066,"€250,000,000 TERM FACILITY AGREEMENT",0,7909066,217889_1,1469
7909067,"€250,000,000",0,7909067,217889_2,1469
7909068,TERM FACILITY AGREEMENT,0,7909068,217889_3,1469
7909069,Dated 8 March 2017 for,0,7909069,217889_4,1469


In [7]:
df['label'].value_counts()

label
0    1252811
1       1544
Name: count, dtype: int64

In [8]:
df['word_count'] = df['sentence'].apply(lambda x: len(str(x).split()))
df.head()

/var/folders/ll/hqtjh2m5481_qkt_4p9g5v940000gn/T/ipykernel_46640/75972406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_count'] = df['sentence'].apply(lambda x: len(str(x).split()))


,sentence,label,index,sent_id,topic_id,word_count
7909065,Exhibit 4.34,0,7909065,217889_0,1469,2
7909066,"€250,000,000 TERM FACILITY AGREEMENT",0,7909066,217889_1,1469,4
7909067,"€250,000,000",0,7909067,217889_2,1469,1
7909068,TERM FACILITY AGREEMENT,0,7909068,217889_3,1469,3
7909069,Dated 8 March 2017 for,0,7909069,217889_4,1469,5


In [9]:
import pandas as pd

def merge_short_sentences(data):
    data = data.dropna()
    # Ensure the data is a DataFrame
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame.")
    
    # Check that the necessary columns exist
    if 'sentence' not in data.columns or 'label' not in data.columns:
        raise ValueError("DataFrame must contain 'sentence' and 'label' columns.")

    # Initialize a list to hold the new rows
    new_rows = []
    # Placeholder for the previous row, if it needs to be merged
    previous_row = None

    for _, row in data.iterrows():
        # If the sentence is short and there is a previous row, merge them
        if len(row['sentence']) <= 50 and previous_row is not None:
            previous_row['sentence'] += " " + row['sentence']
            # Assuming the label merging strategy is to take the label of the first sentence
            # Modify as needed, e.g., combine labels, choose max/min, or handle as a list
        else:
            # If there's a previous row waiting to be added (either merged or not), add it now
            if previous_row is not None:
                new_rows.append(previous_row)
            # Update previous_row to the current row for potential future merging
            previous_row = row.copy()

    # Add the last row if it wasn't added yet
    if previous_row is not None:
        new_rows.append(previous_row)

    # Create a new DataFrame from the merged rows
    return pd.DataFrame(new_rows)

In [10]:
zeros = df[df['label'] ==0][['sentence', 'label']]
zeros.shape, zeros['label'].value_counts()

((1252811, 2),
 label
 0    1252811
 Name: count, dtype: int64)

In [11]:
ones =  df[df['label'] ==1][['sentence', 'label']]
ones.shape, ones['label'].value_counts()

((1544, 2),
 label
 1    1544
 Name: count, dtype: int64)

In [12]:
processed_data_zeros = merge_short_sentences(zeros)
processed_data_zeros['sentence_length'] = processed_data_zeros['sentence'].str.len()
print(processed_data_zeros.shape)
processed_data_zeros.head()

(575010, 3)


,sentence,label,sentence_length
7909065,"Exhibit 4.34 €250,000,000 TERM FACILITY AGREEM...",0,949
7909137,26 . ROLE OF THE AGENT AND THE MANDATED LEAD A...,0,914
7909187,Schedule 13 CONDITIONS PRECEDENT REQUIRED TO B...,0,139
7909189,"( 1 ) LUXOTTICA GROUP S.p.A. , as borrower ( t...",0,102
7909191,"as original guarantor ( the "" Original Guarant...",0,54


In [13]:
processed_data_zeros['sentence_length'].sort_values()

8963154       51
8137057       51
8825153       51
8025536       51
8422972       51
           ...  
8433105    35289
8429771    43786
8502267    51655
8016247    76132
8493973    90922
Name: sentence_length, Length: 575010, dtype: int64

In [14]:
processed_data_ones = merge_short_sentences(ones)
processed_data_ones['sentence_length'] = processed_data_ones['sentence'].str.len()
print(processed_data_ones.shape)
processed_data_ones.head()

(802, 3)


,sentence,label,sentence_length
7912788,"( a ) Each Borrower agrees to pay , in US Doll...",1,618
7912789,"provided , that if any Lender continues to hav...",1,456
7912790,Accrued facility fees shall be payable in arre...,1,320
7912791,provided that any facility fees accruing on th...,1,200
7912792,All facility fees shall be computed on the bas...,1,207


In [15]:
processed_data_zeros = processed_data_zeros.reset_index(drop=True)
processed_data_ones = processed_data_ones.reset_index(drop=True)

In [16]:
combined_data = pd.concat([processed_data_zeros, processed_data_ones]).reset_index(drop =True)
print(combined_data.shape)
combined_data.head()

(575812, 3)


,sentence,label,sentence_length
0,"Exhibit 4.34 €250,000,000 TERM FACILITY AGREEM...",0,949
1,26 . ROLE OF THE AGENT AND THE MANDATED LEAD A...,0,914
2,Schedule 13 CONDITIONS PRECEDENT REQUIRED TO B...,0,139
3,"( 1 ) LUXOTTICA GROUP S.p.A. , as borrower ( t...",0,102
4,"as original guarantor ( the "" Original Guarant...",0,54


In [17]:
final_df = pd.DataFrame(combined_data['sentence']) 
final_df.head()

,sentence
0,"Exhibit 4.34 €250,000,000 TERM FACILITY AGREEM..."
1,26 . ROLE OF THE AGENT AND THE MANDATED LEAD A...
2,Schedule 13 CONDITIONS PRECEDENT REQUIRED TO B...
3,"( 1 ) LUXOTTICA GROUP S.p.A. , as borrower ( t..."
4,"as original guarantor ( the "" Original Guarant..."


In [18]:
processed_data_zeros_sampled = processed_data_zeros[:100]
processed_data_ones_sampled  = processed_data_ones[:100]
processed_data_zeros_sampled.shape, processed_data_ones_sampled.shape

((100, 3), (100, 3))

In [19]:
final_data_2 = pd.concat([processed_data_zeros_sampled, processed_data_ones_sampled]).reset_index(drop = True)
final_data_2 = pd.DataFrame(final_data_2['sentence'])
final_data_2.head()

,sentence
0,"Exhibit 4.34 €250,000,000 TERM FACILITY AGREEM..."
1,26 . ROLE OF THE AGENT AND THE MANDATED LEAD A...
2,Schedule 13 CONDITIONS PRECEDENT REQUIRED TO B...
3,"( 1 ) LUXOTTICA GROUP S.p.A. , as borrower ( t..."
4,"as original guarantor ( the "" Original Guarant..."


## Pre-training

In [66]:
# Create a Hugging Face dataset
dataset = Dataset.from_pandas(final_data_2)

# Split the dataset into training and testing
train_test_split = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [67]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 180
    })
    test: Dataset({
        features: ['sentence'],
        num_rows: 20
    })
})

## Step 3: Set Up the BERT Model for MLM

In [68]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-cased')
model.train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Step 4: Tokenization

In [69]:
from transformers import BertTokenizer, DataCollatorForLanguageModeling

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True,  # Enables MLM
    mlm_probability=0.15  # 15% of tokens will be masked on average
)

In [70]:
def tokenize_function(examples):
    # Assumes your text data is under the key 'sentence'
    return tokenizer(examples['sentence'], truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize_function, batched=True)
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids'])  # 'labels' are not statically set

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [71]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
train_dataloader = DataLoader(dataset['train'], collate_fn=data_collator, batch_size=64)

## Step 5: Define Training Arguments

In [75]:
from transformers import Trainer, TrainingArguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert_pretrained",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    logging_dir='./logs',
    save_strategy="no",  # No checkpoints saved automatically
#     evaluation_strategy="no",  # No evaluation if you don't need it
    report_to="all"
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset['train'],
)

## Step 6: Start Training

In [76]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=45, training_loss=0.022048101160261367, metrics={'train_runtime': 51.7882, 'train_samples_per_second': 3.476, 'train_steps_per_second': 0.869, 'total_flos': 47376022487040.0, 'train_loss': 0.022048101160261367, 'epoch': 1.0})

In [74]:
pwd

'/Users/mdwivedi/Downloads/core-tech'

## MLM accuracy

In [64]:
def compute_mlm_accuracy(model, tokenizer, dataset, device):
    model.eval()
    eval_dataloader = DataLoader(dataset, batch_size=64, collate_fn=data_collator)
    correct_predictions = total_predictions = 0

    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'labels']}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            labels = batch['labels']

            mask = labels != -100  # Ignore padding tokens
            correct_predictions += (predictions[mask] == labels[mask]).sum().item()
            total_predictions += mask.sum().item()

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    model.train()  # Make sure to reset model to training mode
    return accuracy

In [65]:
eval_dataset=dataset['test']
accuracy = compute_mlm_accuracy(model, tokenizer, eval_dataset, model.device)
print(f"MLM Accuracy: {accuracy:.4f}")

MLM Accuracy: 0.6884


## Save the model

In [ ]:
model.save_pretrained("./bert_model_1469_pretrained")
tokenizer.save_pretrained("./bert_model_1469_pretrained")

## NSP

In [20]:
from transformers import BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments
import torch
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
import pandas as pd

In [52]:
# from transformers import DataCollatorForLanguageModeling, BertTokenizer

# # Initialize the data collator
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False
# )

In [21]:
from transformers import BertTokenizer, BertForPreTraining

# Path to your saved model and tokenizer
model_path = "./embedding_model_comparision/bert_model_1469_pretrained_MLM"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForPreTraining.from_pretrained(model_path)

Some weights of BertForPreTraining were not initialized from the model checkpoint at ./embedding_model_comparision/bert_model_1469_pretrained_MLM and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import random

def create_balanced_sentence_pairs(sentences):
    consecutive_pairs = [(sentences[i], sentences[i + 1], 1) for i in range(len(sentences) - 1)]

    # Generating non-consecutive pairs
    non_consecutive_pairs = []
    while len(non_consecutive_pairs) < len(consecutive_pairs):
        i, j = random.sample(range(len(sentences)), 2)
        if abs(i - j) > 1:  # Ensure that the pair is not consecutive
            non_consecutive_pairs.append((sentences[i], sentences[j], 0))

    # Combine and shuffle
    all_pairs = consecutive_pairs + non_consecutive_pairs
    random.shuffle(all_pairs)

    return all_pairs

In [24]:
final_data_2.head()

,sentence
0,"Exhibit 4.34 €250,000,000 TERM FACILITY AGREEM..."
1,26 . ROLE OF THE AGENT AND THE MANDATED LEAD A...
2,Schedule 13 CONDITIONS PRECEDENT REQUIRED TO B...
3,"( 1 ) LUXOTTICA GROUP S.p.A. , as borrower ( t..."
4,"as original guarantor ( the "" Original Guarant..."


In [25]:
train_data = pd.DataFrame(create_balanced_sentence_pairs(final_data_2['sentence']))
train_data.columns = ['sentence_a', 'sentence_b', 'label']
train_data.head()

,sentence_a,sentence_b,label
0,"United Kingdom , any member state of the Europ...",( e ) commercial paper not convertible or exch...,1
1,"Anti-Money Laundering Laws "" means the Executi...",The Borrower agrees to pay to the Administrati...,0
2,"Confidential Information "" means all informati...","( b ) The Borrowers shall pay to the Agent , f...",0
3,( c ) with respect to which an Insolvency Even...,and payment is made within 5 Business Days of ...,1
4,Facility Fee accruing from and after the Closi...,The Unused Facility Fee will be calculated and...,1


In [72]:
from sklearn.model_selection import train_test_split

# Splitting data
train_df, val_df = train_test_split(train_data, test_size=0.1)

In [90]:
from transformers import BertTokenizer, DataCollatorForLanguageModeling

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def tokenize_nsp(examples):
    sentence_a = examples['sentence_a']
    sentence_b = examples['sentence_b']
    # Tokenize each pair and ensure that the total length does not exceed the max_length
    tokenized_input = tokenizer(sentence_a, sentence_b, 
                                padding="max_length",  # Ensures all outputs are padded to max_length
                                truncation=True,       # Ensures truncation to fit the max_length
                                max_length=512,        # Define max_length according to your model capacity
                                return_tensors="pt")
    return tokenized_input


# Create Hugging Face dataset
# Convert to Hugging Face dataset
train_dataset = Dataset.from_pandas(train_df)

# Map tokenization
train_dataset = train_dataset.map(tokenize_nsp, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

# DataLoaders
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)

Map:   0%|          | 0/358 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [91]:
train_dataset

Dataset({
    features: ['sentence_a', 'sentence_b', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 358
})

In [87]:
import torch

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [88]:
training_args = TrainingArguments(
    output_dir='./bert_nsp_trained',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    logging_dir='./logs_nsp',
    save_strategy="no",
    report_to="all"
)

from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs['seq_relationship_logits']
        labels = inputs.get("labels")
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
            return (loss, outputs) if return_outputs else loss
        else:
            return (None, outputs) if return_outputs else None

# Then use CustomTrainer instead of Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     data_collator=data_collator,
    tokenizer=tokenizer
)

In [89]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=90, training_loss=0.4987948947482639, metrics={'train_runtime': 84.4072, 'train_samples_per_second': 4.241, 'train_steps_per_second': 1.066, 'total_flos': 94876855087104.0, 'train_loss': 0.4987948947482639, 'epoch': 1.0})

In [92]:
def tokenize_nsp_1(data):
    inputs = tokenizer.encode_plus(
        data['sentence_a'], data['sentence_b'], 
        add_special_tokens=True, 
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=512
    )
    return {
        'input_ids': inputs['input_ids'].squeeze(0),  # Remove the batch dimension
        'token_type_ids': inputs['token_type_ids'].squeeze(0),
        'attention_mask': inputs['attention_mask'].squeeze(0),
        'next_sentence_label': torch.tensor(data['label'])  # Assuming label is 0 or 1
    }

# Apply tokenization and prepare the DataLoader
val_dataset = Dataset.from_pandas(val_df)
val_dataset = val_dataset.map(tokenize_nsp_1, remove_columns=val_dataset.column_names)
val_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label'])

val_dataloader = DataLoader(val_dataset, batch_size=64)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [93]:
def calculate_nsp_accuracy(model, dataloader):
    model.eval()
    device = next(model.parameters()).device  # Get the device of the model
    correct_preds, num_samples = 0, 0

    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to the correct device
        with torch.no_grad():
            outputs = model(**batch)
            nsp_logits = outputs.seq_relationship_logits  # Correct attribute for NSP predictions
            predictions = torch.argmax(nsp_logits, dim=1)
            correct_preds += (predictions == batch['next_sentence_label']).sum().item()
            num_samples += predictions.size(0)

    accuracy = correct_preds / num_samples if num_samples > 0 else 0
    return accuracy

# Calculate accuracy on validation dataset
val_accuracy = calculate_nsp_accuracy(model, val_dataloader)
print(f"Validation NSP Accuracy: {val_accuracy:.4f}")

Validation NSP Accuracy: 0.6500


In [ ]:
model.save_pretrained("./bert_model_1469_pretrained_nsp")
tokenizer.save_pretrained("./bert_model_1469_pretrained_nsp")

In [1]:
pwd

'/Users/mdwivedi/Downloads/core-tech'

## Finetuning on same topic 1469

In [5]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Load the dataset
print("Loading data")
df = pd.read_csv('due_dilligence_data.csv')
df.head()

,sentence,label,index,sent_id,topic_id
0,EXECUTION VERSION,B,0,150705_0,1252
1,THE BANK OF NOVA SCOTIA,B,1,150705_1,1252
2,"as Sole Bookrunner , Lead Arranger and Adminis...",B,2,150705_2,1252
3,- and-,B,3,150705_3,1252
4,BMO CAPITAL MARKETS,B,4,150705_4,1252


In [7]:
df.head()

,sentence,label,index,sent_id,topic_id
0,EXECUTION VERSION,B,0,150705_0,1252
1,THE BANK OF NOVA SCOTIA,B,1,150705_1,1252
2,"as Sole Bookrunner , Lead Arranger and Adminis...",B,2,150705_2,1252
3,- and-,B,3,150705_3,1252
4,BMO CAPITAL MARKETS,B,4,150705_4,1252


In [10]:
data = df[df['topic_id'] == 1469]
data['label'] = data['label'].replace({'B': 0, '1': 1})
data.head()

/var/folders/ll/hqtjh2m5481_qkt_4p9g5v940000gn/T/ipykernel_1082/952703032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data['label'].replace({'B': 0, '1': 1})


,sentence,label,index,sent_id,topic_id
7909065,Exhibit 4.34,0,7909065,217889_0,1469
7909066,"€250,000,000 TERM FACILITY AGREEMENT",0,7909066,217889_1,1469
7909067,"€250,000,000",0,7909067,217889_2,1469
7909068,TERM FACILITY AGREEMENT,0,7909068,217889_3,1469
7909069,Dated 8 March 2017 for,0,7909069,217889_4,1469


In [13]:
data['sentence'] = data['sentence'].astype(str)

/var/folders/ll/hqtjh2m5481_qkt_4p9g5v940000gn/T/ipykernel_1082/1977311852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentence'] = data['sentence'].astype(str)


In [31]:
train_data, test_data = train_test_split(data, test_size=0.3, stratify=data['label'])

In [34]:
# train_data.label.value_counts(), test_data.label.value_counts()

In [14]:
# Convert the DataFrame to Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
print("Tokenizing...")

# Initialize the tokenizer
tokenizer_path = "./embedding_model_comparision/bert_model_1469_pretrained_nsp"
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [16]:
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [17]:
model_path = "./embedding_model_comparision/bert_model_1469_pretrained_nsp"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./embedding_model_comparision/bert_model_1469_pretrained_nsp and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import torch

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to the appropriate device
model.to(device)

Using device: cpu


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [26]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, np.ndarray):
        predictions = np.argmax(logits, axis=1)
    else:
        predictions = logits.argmax(axis=-1)
        predictions = predictions.detach().cpu().numpy()  # Ensure conversion only if it's a tensor
        labels = labels.detach().cpu().numpy()

    # Calculate the classification report
    full_report = classification_report(labels, predictions, target_names=['Class 0', 'Class 1'], output_dict=True)
    
    return {'full_classification_report': full_report}

In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./model_output',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    fp16=True,
    logging_dir='./logs',
    save_strategy="no",
    report_to="all",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [ ]:
print("Training started")
trainer.train()

In [ ]:
results = trainer.evaluate()
full_report = results['full_classification_report']

print("Full Classification Report:")
for label, metrics in full_report.items():
    if isinstance(metrics, dict):  # This ensures we're only looking at class metrics
        print(f"Label: {label}")
        for metric, value in metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        print()  # Adds a newline for better readability

In [ ]:
model.save_pretrained('./bert_1469_finetuned_classification')
tokenizer.save_pretrained('./bert_1469_finetuned_classification')